In [24]:
import requests
import json

In [ ]:
## Currently if you call a model and its already loaded, it goes through, if the model is not loaded, the model is loaded and then a request gets send to it.
## Requests other than the first are quite fast.
## Bigger models take much longer

BASE_URL = "http://100.118.250.126:6000" # Also worked on: "http://localhost:6000"
GENERATE_URL = f"{BASE_URL}/generate"
GENERATE_STREAM_URL = f"{BASE_URL}/generate_stream"


In [ ]:

def generate_text(model_name, text, max_length=100):
    """
    Calls the /generate endpoint and prints the full generated text.
    """
    payload = {
        "model_name": model_name,
        "text": text,
        "max_length": max_length
    }
    
    response = requests.post(GENERATE_URL, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        print("\n=== /generate ===")
        print(f"Model Name: {model_name}")
        print(f"Prompt: {text}")
        print("Generated Text:")
        print(data.get("response", "No response field in JSON"))
        print(f"Time taken: {data.get('time_taken', 'N/A')} seconds")
    else:
        print(f"Error: {response.status_code} - {response.text}")

def generate_text_stream(model_name, text, max_length=100):
    """
    Calls the /generate_stream endpoint and prints tokens as they arrive.
    """
    payload = {
        "model_name": model_name,
        "text": text,
        "max_length": max_length
    }
    
    response = requests.post(GENERATE_STREAM_URL, json=payload, stream=True)
    
    if response.status_code == 200:
        print("\n=== /generate_stream ===")
        print(f"Model Name: {model_name}")
        print(f"Prompt: {text}")
        print("Streaming Generated Text:")
        for chunk in response.iter_lines(decode_unicode=True):
            if chunk:
                print(chunk, end=" ", flush=True)
        print()  # New line after the stream ends
    else:
        print(f"Error: {response.status_code} - {response.text}")

# Example prompts
prompt1 = "Explain the key differences between tax credits and tax deductions."  
prompt2 = "What are the potential risks and benefits of automating IRS tax processing?"  



| **Model**                              | **Architecture / Basis**                                             | **Estimated Max Token Limit** | **Estimated Max Word Limit**  |
|----------------------------------------|----------------------------------------------------------------------|-------------------------------|-------------------------------|
| Equall/Saul-7B-Instruct-v1             | Continued pretraining of Mistral‑7B                                  | ~8,192 tokens                 | ~6,000 words                  |
| ricdomolm/lawma-8b                     | Fine‑tuned on Llama‑3 8B Instruct                                    | ~8,192 tokens                 | ~6,000 words                  |
| ricdomolm/lawma-70b                    | Fine‑tuned on Llama‑3 70B Instruct for legal classification tasks¹    | ~4,096 tokens                 | ~3,072 words                  |
| deepseek‑ai/DeepSeek‑V2‑Lite (and Chat)  | Mixture‑of‑Experts (MoE) model with an extended context window       | 32,000 tokens                 | ~24,000 words                 |



¹ Although the base Llama‑3‑70B model supports an 8k-token context, this version was configured for legal classification—where shorter inputs (and corresponding outputs) are sufficient—resulting in an effective limit of approximately 4k tokens.

Also the code run until this was:

```
python3 app.py 
 * Serving Flask app 'app'
 * Debug mode: off
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://100.118.250.126:6000

```

In [13]:
# Test: Using "lawma_70b" model. This is a large model.
generate_text("lawma_70b", prompt1, max_length=100)
generate_text_stream("lawma_70b", prompt2, max_length=100)


=== /generate ===
Model Name: lawma_70b
Prompt: Explain the key differences between tax credits and tax deductions.
Generated Text:
You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:
Explain the key differences between tax credits and tax deductions. 't use legal jargon.
It seems like tax credits and tax deductions both reduce your tax bill, but I've heard they work differently.  I'd love to understand the differences.
Answer: I'd be happy to help you with that!

Tax credits and tax deductions both reduce your tax bill, but they work in different ways.

Tax DEDUCTIONS:

1. Reduce taxable income: DEDUCTIONS lower your taxable income, which means you're reducing the amount of income that's ...
Time taken: 4780.952471971512 seconds

=== /generate_stream ===
Model Name: lawma_70b
Prompt: What are the potential risks and benefits of automating IRS tax processing?
Streaming Generated Text:
  


Here are the logs for the above cell:


```
[load_model] Attempting to load 'lawma_70b' onto cuda:1 ...
2025-02-17 10:04:00.794940: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 10:04:01.810556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|█████████████| 30/30 [02:31<00:00,  5.04s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
[load_model] 'lawma_70b' loaded on cuda:1. Current allocated: 39168.75 MB
172.24.9.126 - - [17/Feb/2025 10:35:00] "POST /generate HTTP/1.1" 200 -
172.24.9.126 - - [17/Feb/2025 10:35:16] "POST /generate_stream HTTP/1.1" 200 

```

In [15]:
# Running again to check if its an end point issue

generate_text_stream("lawma_70b", prompt2, max_length=100)


=== /generate_stream ===
Model Name: lawma_70b
Prompt: What are the potential risks and benefits of automating IRS tax processing?
Streaming Generated Text:
  Answer: K 


Seems like there is an issue with the stream answers with 70b. 


Also, here are the logs:


```

[load_model] Attempting to load 'lawma_70b' ...
2025-02-17 21:51:20.115131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 21:51:21.135897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:50<00:00,  1.67s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
[load_model] 'lawma_70b' loaded. It has been automatically partitioned across available GPUs.
/apps/default-python/lib/python3.10/site-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
172.24.9.126 - - [17/Feb/2025 21:52:30] "POST /generate_stream HTTP/1.1" 200 -

```

In [12]:
# Test: Using "saul_7b_instruct" model
generate_text("saul_7b_instruct", prompt1, max_length=50)
generate_text_stream("saul_7b_instruct", prompt2, max_length=50)


=== /generate ===
Model Name: saul_7b_instruct
Prompt: Explain the key differences between tax credits and tax deductions.
Generated Text:
You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:
Explain the key differences between tax credits and tax deductions.
Answer: [/INST] Tax credits and tax deductions are two important tools used to reduce a person's taxable income. However, they work in different ways and have distinct characteristics.

A tax credit directly reduces your ...
Time taken: 2.0661349296569824 seconds

=== /generate_stream ===
Model Name: saul_7b_instruct
Prompt: What are the potential risks and benefits of automating IRS tax processing?
Streaming Generated Text:
What are potential risks and benefits of this approach to handling tax returns? [/INST] Automating IRS tax processing can provide several benefits and risks: Benefits: 1. Improved efficiency - Automation can help process 


Here are the logs for the above cell:


```

[load_model] Attempting to load 'saul_7b_instruct' ...
Loading checkpoint shards: 100%|███████████████| 6/6 [01:12<00:00, 12.11s/it]
[load_model] 'saul_7b_instruct' loaded. It has been automatically partitioned across available GPUs.
/apps/default-python/lib/python3.10/site-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
172.24.9.126 - - [17/Feb/2025 10:59:03] "POST /generate HTTP/1.1" 200 -
172.24.9.126 - - [17/Feb/2025 10:59:04] "POST /generate_stream HTTP/1.1" 200 


```


In [11]:
# Test: Using "lawma_8b" model
generate_text("lawma_8b", prompt1, max_length=50)
generate_text_stream("lawma_8b", prompt2, max_length=50)



=== /generate ===
Model Name: lawma_8b
Prompt: Explain the key differences between tax credits and tax deductions.
Generated Text:
You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:
Explain the key differences between tax credits and tax deductions. A tax credit reduces 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
Time taken: 2.511077404022217 seconds

=== /generate_stream ===
Model Name: lawma_8b
Prompt: What are the potential risks and benefits of automating IRS tax processing?
Streaming Generated Text:
 A B C D E F G H I J E F G H I J E F G H I J E F G H I J E F G H I J E F G H I J E F G H I J E F G H 


Here are the logs for the above cell:


```
[load_model] Attempting to load 'lawma_8b' ...
Loading checkpoint shards: 100%|███████████████| 4/4 [00:37<00:00,  9.28s/it]
[load_model] 'lawma_8b' loaded. It has been automatically partitioned across available GPUs.
/apps/default-python/lib/python3.10/site-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
172.24.9.126 - - [17/Feb/2025 10:57:45] "POST /generate HTTP/1.1" 200 -
172.24.9.126 - - [17/Feb/2025 10:57:46] "POST /generate_stream HTTP/1.1" 200 

```

In [10]:
# Test: Using "DeepSeek-V2-Lite" model
generate_text("DeepSeek-V2-Lite", prompt2, max_length=200)
generate_text_stream("DeepSeek-V2-Lite", prompt1, max_length=200)


=== /generate ===
Model Name: DeepSeek-V2-Lite
Prompt: What are the potential risks and benefits of automating IRS tax processing?
Generated Text:
You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:
What are the potential risks and benefits of automating IRS tax processing?

Automating IRS tax processing can have several potential risks and benefits. Here are some of them:

Risks:
1. Data Security: Automated systems can be vulnerable to cyber-attacks, leading to unauthorized access to sensitive taxpayer information.
2. System Errors: Automation can lead to errors in processing, such as incorrect calculations or misinterpretation of tax codes.
3. Dependence on Technology: Over-reliance on automated systems can make the IRS vulnerable to technical failures or outages.
4. Training and Support: Staff may require additional training to effectively use new automated systems, which can be costly and time-consuming.

Benefits:
1. Efficiency: A

Here are the logs for the above cell:


```

[load_model] Attempting to load 'DeepSeek-V2-Lite' ...
2025-02-17 10:53:39.516528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 10:53:40.689715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|███████████████| 4/4 [00:22<00:00,  5.56s/it]
[load_model] 'DeepSeek-V2-Lite' loaded. It has been automatically partitioned across available GPUs.
/apps/default-python/lib/python3.10/site-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
172.24.9.126 - - [17/Feb/2025 10:54:36] "POST /generate HTTP/1.1" 200 -
172.24.9.126 - - [17/Feb/2025 10:54:37] "POST /generate_stream HTTP/1.1" 200

```

After this, we: 

1. Introduced a shared `perform_generation` helper function to centralize text generation logic for all endpoints.  
2. Added new endpoints (`/v1/models`, `/v1/chat/completions`, and `/v1/completions`) that leverage this helper to ensure consistent responses and reduced code duplication. This was to align us with the testing we were doing using LM Studio. 

In [22]:
MODELS_URL = f"{BASE_URL}/v1/models"
CHAT_COMPLETIONS_URL = f"{BASE_URL}/v1/chat/completions"
COMPLETIONS_URL = f"{BASE_URL}/v1/completions"

In [27]:
def test_models():
    """
    Test the GET /v1/models endpoint.
    Prints:
      - The status code.
      - A summary list of available models.
      - The full JSON response.
    """
    response = requests.get(MODELS_URL)
    print("\n=== GET /v1/models ===")
    print("Status Code:", response.status_code)
    try:
        data = response.json()
        print("\nAvailable Models:")
        for model in data.get("data", []):
            print(f" - ID: {model.get('id')}, Object: {model.get('object')}")
        print("\nFull Response JSON:")
        print(json.dumps(data, indent=2))
    except Exception as e:
        print("Error parsing JSON:", e)

test_models()



=== GET /v1/models ===
Status Code: 200

Available Models:
 - ID: saul_7b_instruct, Object: model
 - ID: lawma_8b, Object: model
 - ID: lawma_70b, Object: model
 - ID: DeepSeek-V2-Lite, Object: model

Full Response JSON:
{
  "data": [
    {
      "id": "saul_7b_instruct",
      "object": "model"
    },
    {
      "id": "lawma_8b",
      "object": "model"
    },
    {
      "id": "lawma_70b",
      "object": "model"
    },
    {
      "id": "DeepSeek-V2-Lite",
      "object": "model"
    }
  ],
  "object": "list"
}


In [28]:

def test_chat_completions_deepseek():
    """
    Test the POST /v1/chat/completions endpoint using the DeepSeek-V2-Lite model.
    Prints:
      - The status code.
      - A summary of the generated chat response.
      - The full JSON response.
    """
    payload = {
        "model": "DeepSeek-V2-Lite",
        "messages": [
            {"role": "system", "content": "You are a helpful tax and legal advisor."},
            {"role": "user", "content": "What are the benefits of tax deductions?"}
        ],
        "max_tokens": 50
    }
    response = requests.post(CHAT_COMPLETIONS_URL, json=payload)
    print("\n=== POST /v1/chat/completions ===")
    print("Status Code:", response.status_code)
    try:
        data = response.json()
        # Extract key details from the response
        choice = data.get("choices", [{}])[0]
        message = choice.get("message", {})
        print("\nChat Completion Summary:")
        print(f" - Model: {data.get('model')}")
        print(f" - Generated Response: {message.get('content')}")
        usage = data.get("usage", {})
        print(f" - Prompt Tokens: {usage.get('prompt_tokens')}")
        print(f" - Completion Tokens: {usage.get('completion_tokens')}")
        print("\nFull Response JSON:")
        print(json.dumps(data, indent=2))
    except Exception as e:
        print("Error parsing JSON:", e)




test_chat_completions_deepseek()



=== POST /v1/chat/completions ===
Status Code: 200

Chat Completion Summary:
 - Model: DeepSeek-V2-Lite
 - Generated Response: You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:
What are the benefits of tax deductions?

As a tax and legal advisor, I would explain that tax deductions are a valuable tool for individuals and businesses to reduce their taxable income. Here are some of the key benefits of tax deductions:

1. **Reduction in Taxable ...
 - Prompt Tokens: 30
 - Completion Tokens: 81

Full Response JSON:
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "You are a helpful tax and legal advisor. Answer the following question in a clear and concise manner:\nWhat are the benefits of tax deductions?\n\nAs a tax and legal advisor, I would explain that tax deductions are a valuable tool for individuals and businesses to reduce their taxable income. Here are some of the key 

In [29]:
def test_completions_deepseek():
    """
    Test the POST /v1/completions endpoint using the DeepSeek-V2-Lite model.
    Prints:
      - The status code.
      - A summary of the generated text completion.
      - The full JSON response.
    """
    payload = {
        "model": "DeepSeek-V2-Lite",
        "prompt": "Explain the key differences between tax credits and tax deductions.",
        "max_tokens": 50
    }
    response = requests.post(COMPLETIONS_URL, json=payload)
    print("\n=== POST /v1/completions ===")
    print("Status Code:", response.status_code)
    try:
        data = response.json()
        choice = data.get("choices", [{}])[0]
        print("\nText Completion Summary:")
        print(f" - Model: {data.get('model')}")
        print(f" - Generated Text: {choice.get('text')}")
        usage = data.get("usage", {})
        print(f" - Prompt Tokens: {usage.get('prompt_tokens')}")
        print(f" - Completion Tokens: {usage.get('completion_tokens')}")
        print("\nFull Response JSON:")
        print(json.dumps(data, indent=2))
    except Exception as e:
        print("Error parsing JSON:", e)
    
test_completions_deepseek()


=== POST /v1/completions ===
Status Code: 200

Text Completion Summary:
 - Model: DeepSeek-V2-Lite
 - Generated Text: Explain the key differences between tax credits and tax deductions.

Tax credits and tax deductions are both tools used by the government to reduce the amount of tax that individuals and businesses owe. However, there are some key differences between the two:

1. Tax Credits:
   a. Tax credits ...
 - Prompt Tokens: 12
 - Completion Tokens: 63

Full Response JSON:
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "text": "Explain the key differences between tax credits and tax deductions.\n\nTax credits and tax deductions are both tools used by the government to reduce the amount of tax that individuals and businesses owe. However, there are some key differences between the two:\n\n1. Tax Credits:\n   a. Tax credits ..."
    }
  ],
  "created": 1739830505,
  "id": "cmpl-1739830505",
  "model": "DeepSeek-V2-Lite",
  "object": "text_completion"

## Pinecone End Point

The new `/search` API endpoint enables querying a Pinecone index to retrieve the most relevant PDF chunks based on an input query, utilizing embedding models for semantic search. for a couple of seconds
This new API endpoint allows users to search IRS document embeddings by encoding their query and returning the most relevant text chunks from a Pinecone index.

In [34]:
import os
import requests
import json

def test_search_endpoint(query, top_k=5):
    payload = {"query": query, "top_k": top_k}
    
    response = requests.post(f"{BASE_URL}/search", json=payload)

    print("=== POST /search ===")
    print("Status Code:", response.status_code)

    try:
        data = response.json()
    except Exception as e:
        print("Error parsing JSON response:", e)
        return

    # Print a nicely formatted summary.
    print("\nSearch Query:")
    print(f"  {data.get('query')}")

    print("\nTop Results:")
    for i, result in enumerate(data.get("results", []), start=1):
        print(f" {i}. Score: {result.get('score'):.4f}")
        print(f"    Text: {result.get('text')}\n")

    # Print the full JSON response.
    print("Full JSON Response:")
    print(json.dumps(data, indent=2))

# Example usage:
test_query = "Which forms do I need for self-employment?"
test_search_endpoint(test_query)


=== POST /search ===
Status Code: 200

Search Query:
  Which forms do I need for self-employment?

Top Results:
 1. Score: 0.6047
    Text: self-employment income from separate nonfarm or farm
businesses, each of you must complete and file a
separate Schedule C (Form 1040) or Schedule F (Form
1040). Be sure to enter at the top of each Schedule C
(Form 1040) or Schedule F (Form 1040) the name and
SSN of the spouse who owns the business. Each of you
must also complete a separate Schedule SE (Form 1040).
Attach these pages to a single Form 1040-SS.
Business Owned and Operated by
Spouses

 2. Score: 0.5987
    Text: Schedule SE (Form 1040), Self-Employment Tax, to complete your return.
You may only need to file Form 1040-SS and none of the schedules. However, if your return is more complicated (for
example, you claim certain deductions or credits or owe additional taxes), you will need to complete one or more of the
schedules. Below is a general guide to which schedule(s) you will need to 

## Integration test of prompt with pinecone

In [42]:
import requests
import json

# Define the endpoint path for the /v1/completions API
COMPLETIONS_URL = f"{BASE_URL}/v1/completions"

def test_completions_deepseek(tax_optimize=True):
    """
    Test the POST /v1/completions endpoint using the DeepSeek-V2-Lite model.
    Test with tax optimization enabled or disabled.
    
    Parameters:
    - tax_optimize (bool): Flag to use tax optimization in the request.

    Prints:
      - The status code.
      - A summary of the generated text completion.
      - The full JSON response.
    """
    payload = {
        "model": "DeepSeek-V2-Lite",
        "prompt": "Explain the key differences between tax credits and tax deductions.",
        "max_tokens": 2000,
        "tax_optimize": tax_optimize,
        "n": 2  # Number of search results to include in context
    }
    response = requests.post(COMPLETIONS_URL, json=payload)
    print("\n=== POST /v1/completions ===")
    print("Status Code:", response.status_code)
    try:
        data = response.json()
        choice = data.get("choices", [{}])[0]
        print("\nText Completion Summary:")
        print(f" - Model: {data.get('model')}")
        print(f" - prompt: {payload.get('prompt')}")
        print(f" - Generated Text: \n{choice.get('text')}")
        usage = data.get("usage", {})
        print(f" - Prompt Tokens: {usage.get('prompt_tokens')}")
        print(f" - Completion Tokens: {usage.get('completion_tokens')}")
        print("\nFull Response JSON:")
        print(json.dumps(data, indent=2))
    except Exception as e:
        print("Error parsing JSON:", e)

# Test with tax optimization enabled
test_completions_deepseek(tax_optimize=True)




=== POST /v1/completions ===
Status Code: 200

Text Completion Summary:
 - Model: DeepSeek-V2-Lite
 - prompt: Explain the key differences between tax credits and tax deductions.
 - Generated Text: 
As a tax advisor and legal expert, the key differences between tax credits and tax deductions are:

1. Nature of Reduction: Tax deductions reduce the taxable income, which in turn lowers the amount of tax owed. Tax credits directly reduce the tax amount owed on a dollar-for-dollar basis.

2. Effect on Tax Liability: Tax deductions reduce the tax rate, while tax credits directly reduce the tax liability.

3. Availability and Use: Tax deductions are available to all taxpayers, regardless of their income level. Tax credits are often refundable, meaning they can provide a refund even if the credit exceeds the tax liability.

4. Application: Tax deductions are subtracted from the total income to calculate taxable income. Tax credits are subtracted directly from the tax liability.

5. Impact on N

In [39]:
# Test with tax optimization disabled
test_completions_deepseek(tax_optimize=False)


=== POST /v1/completions ===
Status Code: 200

Text Completion Summary:
 - Model: DeepSeek-V2-Lite
 - Generated Text: As a tax and legal advisor, it is important to understand the nuances between tax credits and tax deductions, as they are both valuable tools for reducing taxable income. Here are the key differences between the two:

1. Nature of Benefit:
   - Tax Credits: A tax credit is a dollar-for-dollar reduction in the amount of income tax you owe. It is more beneficial than a tax deduction because it directly reduces the tax liability. Credits can be either nonrefundable or refundable. Nonrefundable credits can only reduce your tax liability to zero, while refundable credits can result in a tax refund even if your tax liability is less than the credit amount.
   - Tax Deductions: A tax deduction reduces the amount of your income that is subject to tax. It is a dollar reduction in your taxable income, which in turn reduces the amount of tax you owe. Deductions are generally nonr